# Sprint 6 · Webinar 18 · Data Analytics teórico (Preparación de datos, distribuciones y GitHub)

## Fecha

- **Fecha:** ____ / ____ / 2025
- **Duración:** 100 minutos
- **Modalidad:** Teórica + demostración guiada (hands-on)


## Objetivos de la sesión teórica

Al finalizar la sesión, las y los estudiantes podrán:

1. Preparar datos para análisis estadístico identificando variables relevantes, tipos de datos y reglas de calidad.
2. Detectar y manejar valores ausentes o inválidos usando estrategias apropiadas según el contexto de negocio.
3. Automatizar tareas repetitivas de limpieza con funciones, bucles e iterables en Python.
4. Describir y comparar distribuciones numéricas y categóricas con medidas estadísticas y visualizaciones básicas.
5. Interpretar distribuciones desde una perspectiva de negocio (sesgos, outliers, estacionalidad, segmentos).
6. Ejecutar un flujo mínimo de trabajo con Git y GitHub: instalar, crear repositorio, clonar y realizar el primer commit.


## Agenda sugerida (100 minutos)

| Minutos | Tema | Resultado esperado |
|---:|---|---|
| 0–10 | **Ejercicio 0 (breakout):** ¿Qué significa “datos listos para estadística”? | Criterios compartidos de calidad y variables relevantes |
| 10–35 | **7.1 Preparación de datos:** variables, nulos/ inválidos, automatización | Checklist + primeros patrones de limpieza |
| 35–75 | **7.2 Distribuciones:** medidas y visualizaciones (histograma / boxplot) | Lectura crítica de distribuciones y outliers |
| 75–95 | **Hands-on lab GitHub:** instalación, cuenta, repo, clone, commit | Repo creado + commit realizado |
| 95–100 | Cierre + próximos pasos | Conexión con el siguiente webinar/práctica |


## Ejercicio 0 · Calentamiento en breakout rooms (discusión conceptual, 10 min)

En grupos pequeños, respondan:

1. ¿Qué condiciones mínimas debe cumplir un dataset para que tenga sentido calcular medias, medianas o correlaciones?
2. Si una columna tiene valores “N/A”, “-”, “?”, “desconocido”, ¿eso es igual a **NaN**? ¿Por qué importa?
3. Den un ejemplo de un valor **inválido** (no faltante) en un contexto de negocio (p. ej., edad negativa, fecha futura, monto negativo).

**Entrega rápida (en 1 minuto):** 2 criterios de “calidad” y 1 regla de validación que usarían.


## Ejercicio 1 · Preparación de datos para análisis estadístico (7.1)

En esta parte construiremos y exploraremos un dataset sintético (similar a un extracto transaccional / ventas) que incluye:

- Variables numéricas y categóricas
- Valores ausentes (missing)
- Valores inválidos (p. ej., negativos, categorías no permitidas)
- Inconsistencias de formato (strings, símbolos, separadores)

**Enfoque:** antes de “calcular métricas”, primero definimos qué columnas importan y qué reglas de calidad aplican.

### 7.1.1 Identificando variables relevantes para el análisis

- ¿Qué variable(s) serían tu *target* o variable de interés?
- ¿Qué variables explicarían diferencias (segmentación)?
- ¿Qué variables podrían ser ruido o irrelevantes?

### 7.1.2 Manejando valores ausentes o inválidos

- Diferenciar **faltante** vs **inválido**
- Estrategias: eliminación, imputación simple, imputación condicionada, marcadores (flags)

### 7.1.3 Funciones de Python para automatizar la limpieza de datos

- Reglas reusables y testeables
- “Una función por transformación” (pequeñas y composables)



In [1]:
import pandas as pd
import numpy as np

# Dataset sintético para la sesión (sin dependencias externas)
rng = np.random.default_rng(42)
n = 250

df = pd.DataFrame({
    "order_id": np.arange(1, n+1),
    "segmento": rng.choice(["retail", "pyme", "enterprise", "N/A", ""], size=n, p=[0.45, 0.30, 0.20, 0.03, 0.02]),
    "pais": rng.choice(["CO", "MX", "PE", "CL", "AR", "??"], size=n, p=[0.35, 0.25, 0.12, 0.10, 0.10, 0.08]),
    "monto": rng.normal(loc=180_000, scale=75_000, size=n).round(0),
    "descuento_pct": rng.choice([0, 5, 10, 15, 20, None], size=n, p=[0.30, 0.25, 0.20, 0.12, 0.08, 0.05]),
    "metodo_pago": rng.choice(["tarjeta", "transferencia", "efectivo", "cripto", None], size=n, p=[0.55, 0.20, 0.18, 0.02, 0.05]),
    "dias_entrega": rng.normal(loc=3.5, scale=1.8, size=n).round(1),
})

# Inyectar valores inválidos deliberados
idx_neg = rng.choice(df.index, size=12, replace=False)
df.loc[idx_neg, "monto"] = df.loc[idx_neg, "monto"] * -1  # montos negativos inválidos

idx_future = rng.choice(df.index, size=10, replace=False)
df["fecha_compra"] = pd.Timestamp("2025-10-01") + pd.to_timedelta(rng.integers(-60, 30, size=n), unit="D")
df.loc[idx_future, "fecha_compra"] = pd.Timestamp("2030-01-01")  # fecha futura sospechosa

# Forzar mezclas de tipos/formato en 'monto'
idx_money_str = rng.choice(df.index, size=10, replace=False)
df.loc[idx_money_str, "monto"] = df.loc[idx_money_str, "monto"].astype(int).astype(str) + " COP"

df.head()

C:\Users\ljpca\AppData\Local\Temp\ipykernel_22640\3440170527.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['107239 COP' '161061 COP' '11117 COP' '146358 COP' '34737 COP'
 '126459 COP' '158533 COP' '238863 COP' '163385 COP' '91754 COP']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx_money_str, "monto"] = df.loc[idx_money_str, "monto"].astype(int).astype(str) + " COP"


,order_id,segmento,pais,monto,descuento_pct,metodo_pago,dias_entrega,fecha_compra
0,1,enterprise,PE,287491.0,0,transferencia,4.4,2025-08-23
1,2,retail,AR,186864.0,0,tarjeta,4.8,2025-10-11
2,3,enterprise,MX,223558.0,0,None,2.5,2025-09-23
3,4,pyme,CO,175741.0,20,tarjeta,4.5,2025-10-08
4,5,retail,CO,167219.0,0,tarjeta,4.5,2025-08-11


In [2]:
# Información general del DataFrame
df.info()

# Vista rápida de nulos y valores potencialmente problemáticos
resumen_calidad = pd.DataFrame({
    "nulos": df.isna().sum(),
    "pct_nulos": (df.isna().mean() * 100).round(2),
    "n_unicos": df.nunique(dropna=False)
}).sort_values("pct_nulos", ascending=False)

resumen_calidad

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       250 non-null    int32         
 1   segmento       250 non-null    object        
 2   pais           250 non-null    object        
 3   monto          250 non-null    object        
 4   descuento_pct  235 non-null    object        
 5   metodo_pago    237 non-null    object        
 6   dias_entrega   250 non-null    float64       
 7   fecha_compra   250 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int32(1), object(5)
memory usage: 14.8+ KB


,nulos,pct_nulos,n_unicos
descuento_pct,15,6.0,6
metodo_pago,13,5.2,5
order_id,0,0.0,250
segmento,0,0.0,5
pais,0,0.0,6
monto,0,0.0,249
dias_entrega,0,0.0,81
fecha_compra,0,0.0,85


### Preguntas guiadas para el Ejercicio 1

1. ¿Qué columnas parecen “listas” para estadística (numéricas limpias) y cuáles no?
2. ¿Qué problemas de calidad detectas? Clasifícalos como:
   - faltantes (missing)
   - inválidos (fuera de rango / no permitidos)
   - inconsistencias de formato / tipo
3. Si tu análisis fuera “monto promedio por país”, ¿qué columnas serían **críticas**?
4. Propón una regla simple para cada columna:
   - `monto`
   - `descuento_pct`
   - `dias_entrega`
   - `segmento` / `pais`


## Ejercicio 2 · Automatizando la limpieza (7.1.4–7.1.5)

En este ejercicio trabajaremos dos habilidades:

### 7.1.4 Bucles (for & while) e iterables en una colección

- Iterar listas de columnas para aplicar validaciones repetibles
- Construir reportes de calidad por columna

### 7.1.5 Escribiendo tus primeras mini-funciones de limpieza de datos

- Transformaciones pequeñas y reusables:
  - normalización de strings
  - parseo de moneda
  - reglas de rango / dominio
  - banderas de calidad (quality flags)

**Objetivo:** dejar `df_limpio` listo para describir distribuciones en el Ejercicio 3.


In [ ]:
import re

def normalizar_texto(x):
    """Normaliza strings: trim, lower, convierte vacíos/'N/A' a NaN."""
    if pd.isna(x):
        return np.nan
    x = str(x).strip().lower()
    if x in {"", "n/a", "na", "none", "null", "nan"}:
        return np.nan
    return x

def parsear_monto(x):
    """Convierte montos tipo '12345 COP' o '12,345' a float. Devuelve NaN si no es parseable."""
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x)
    s = re.sub(r"[^0-9\-\.]", "", s)  # deja dígitos, '-' y '.'
    try:
        return float(s)
    except ValueError:
        return np.nan

def clip_rango(s: pd.Series, minimo=None, maximo=None):
    """Recorta valores fuera de rango a NaN (estrategia conservadora)."""
    out = s.copy()
    if minimo is not None:
        out = out.where(out >= minimo, np.nan)
    if maximo is not None:
        out = out.where(out <= maximo, np.nan)
    return out

# Copia para limpieza
df_limpio = df.copy()

# Normalización de categóricas (loop sobre columnas)
for col in ["segmento", "pais", "metodo_pago"]:
    df_limpio[col] = df_limpio[col].map(normalizar_texto)

# Parseo de monto
df_limpio["monto"] = df_limpio["monto"].map(parsear_monto)

# Reglas de negocio (ejemplo)
df_limpio["monto"] = clip_rango(df_limpio["monto"], minimo=0)  # montos negativos no permitidos
df_limpio["descuento_pct"] = clip_rango(df_limpio["descuento_pct"].astype(float), minimo=0, maximo=80)
df_limpio["dias_entrega"] = clip_rango(df_limpio["dias_entrega"].astype(float), minimo=0, maximo=30)

# Fecha futura: marcar como NaN para análisis (o crear flag si se requiere auditoría)
hoy = pd.Timestamp("2025-12-14")
df_limpio["fecha_compra"] = pd.to_datetime(df_limpio["fecha_compra"], errors="coerce")
df_limpio.loc[df_limpio["fecha_compra"] > hoy, "fecha_compra"] = pd.NaT

# Flags de calidad (útil para trazabilidad)
df_limpio["flag_monto_parseado"] = df["monto"].apply(lambda x: not pd.isna(parsear_monto(x)))
df_limpio["flag_monto_valido"] = df_limpio["monto"].notna()

df_limpio.head()

## Ejercicio 3 · Describiendo distribuciones de datos (7.2)

Con el dataset ya más consistente, ahora sí describimos distribuciones y las interpretamos.

### 7.2.1 Medidas Estadísticas en Columnas Numéricas

- `count`, `mean`, `median`, `std`, percentiles (p25, p75), `min`, `max`

### 7.2.2 Medidas Estadísticas en Columnas Categóricas

- Frecuencias, proporciones, “top-k”, categorías raras

### 7.2.3 Visualizando distribuciones con Histogramas
### 7.2.4 Visualizando distribuciones con Boxplot

### 7.2.5 Entendiendo el significado de negocios de distribuciones

- Skewness: ¿cola larga por pocos clientes “grandes”?
- Outliers: ¿errores, fraude, clientes VIP, promociones?
- Segmentación: ¿la distribución cambia por país o segmento?



In [ ]:
from IPython.display import display

# Medidas descriptivas para columnas numéricas clave
cols_num = ["monto", "descuento_pct", "dias_entrega"]
display(df_limpio[cols_num].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9]).T)

# Distribuciones categóricas (frecuencia y proporción)
cols_cat = ["segmento", "pais", "metodo_pago"]
for col in cols_cat:
    display(pd.DataFrame({
        "frecuencia": df_limpio[col].value_counts(dropna=False),
        "proporcion": (df_limpio[col].value_counts(dropna=False, normalize=True) * 100).round(2)
    }))

In [ ]:
import matplotlib.pyplot as plt

# Histograma: monto (nota: se ignoran NaN automáticamente)
plt.figure(figsize=(8,4))
plt.hist(df_limpio["monto"].dropna(), bins=25)
plt.title("Distribución de monto (después de limpieza)")
plt.xlabel("monto")
plt.ylabel("frecuencia")
plt.show()

# Boxplot: monto por segmento (requiere filtrar NaN)
plt.figure(figsize=(8,4))
data = [df_limpio.loc[df_limpio["segmento"] == seg, "monto"].dropna() for seg in df_limpio["segmento"].dropna().unique()]
plt.boxplot(data, labels=df_limpio["segmento"].dropna().unique(), vert=True)
plt.title("Monto por segmento (boxplot)")
plt.ylabel("monto")
plt.show()

### Preguntas de reflexión sobre el Ejercicio 3

1. ¿La distribución de `monto` parece simétrica o sesgada? ¿Qué implicación tendría para elegir media vs mediana?
2. Identifica outliers en el boxplot. ¿Los tratarías como errores o como casos de negocio relevantes?
3. ¿Qué segmento/pais muestra mayor variabilidad? ¿Qué hipótesis de negocio podrías plantear?
4. Si tuvieras que reportar un “monto típico” para liderazgo, ¿qué estadístico usarías y por qué?


## 6. Take aways de la sesión teórica

- Preparar datos para estadística no es “rellenar NaN por defecto”: primero se define **propósito** y **reglas de negocio**.
- Diferenciar faltantes vs inválidos evita decisiones erróneas (p. ej., promediar valores imposibles).
- Automatizar con mini-funciones y bucles reduce errores, mejora trazabilidad y facilita reutilización.
- Las distribuciones cuentan una historia: sesgos, colas largas y outliers son señales (no solo “problemas”).
- En análisis práctico, complementa medidas numéricas con visualizaciones; suelen revelar comportamientos que `describe()` no muestra.
- Git/GitHub te permite versionar tu trabajo analítico: reproducibilidad, auditoría y colaboración.


## 7. Cierre y próximos pasos

1. Revisa tu notebook: ¿puedes explicar claramente qué reglas de limpieza aplicaste y por qué?
2. Para el siguiente webinar/práctica:
   - Trae una lista de 5 reglas de calidad que aplicarías a un dataset real (del trabajo o un proyecto personal).
   - Piensa en 2 preguntas de negocio que se respondan con distribuciones (p. ej., tiempos de entrega, montos, churn).

**Indicador de dominio:** puedes tomar un dataset “sucio”, definir reglas explícitas y producir un dataset listo para análisis descriptivo sin perder trazabilidad.


## 8. Hands on lab GitHub (Windows/Mac) + recursos

### Objetivo del laboratorio (20 min)
Instalar Git, crear una cuenta en GitHub, crear un repositorio, clonarlo localmente y realizar el primer commit.

---

### A) Instalar Git

**Windows**
1. Descargar e instalar *Git for Windows*.
2. Verificar instalación en PowerShell:
   - `git --version`

**macOS**
1. Opción 1: instalar *Xcode Command Line Tools*:
   - `xcode-select --install`
2. Verificar:
   - `git --version`

> Nota: si el comando `git` no se reconoce, reinicia la terminal después de instalar.

---

### B) Crear cuenta en GitHub y configurar identidad local

1. Crear cuenta en GitHub (si no existe).
2. En tu terminal, configurar nombre y correo (el mismo correo de GitHub o uno válido):
```bash
git config --global user.name "Tu Nombre"
git config --global user.email "tu_correo@dominio.com"
git config --global init.defaultBranch main
```

Opcional (recomendado): revisar configuración
```bash
git config --global --list
```

---

### C) Crear repositorio y clonarlo

1. En GitHub: **New repository**
   - Nombre sugerido: `webinar18-analisis-estadistico`
   - Inicializar con README (recomendado)

2. Clonar:
```bash
git clone <URL_DEL_REPO>
cd webinar18-analisis-estadistico
```

---

### D) Primer commit (agregar el notebook)

1. Copia tu archivo `.ipynb` dentro del repositorio (carpeta actual).
2. Ver estado:
```bash
git status
```
3. Agregar y confirmar:
```bash
git add .
git commit -m "Agrega notebook Webinar 18: limpieza y distribuciones"
```
4. Subir a GitHub:
```bash
git push -u origin main
```

---

### E) Recursos complementarios

- Documentación oficial de Git: conceptos y comandos básicos
- Guía de GitHub: repos, commits, branches y pull requests
- Recomendación práctica: crea un repositorio por sprint / proyecto y versiona tus notebooks y datasets.
